In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, Sequential

import numpy as np

In [2]:
data_train = np.load('Dataset5_raw_train.npz')
image_train, image_label_train = data_train['image'].astype(np.float16)/ 255.0, data_train['image_label'].astype(np.int8)

KeyboardInterrupt: 

In [ ]:
x_train = np.concatenate([image_train[:2326], image_train[2512:4838], image_train[6119:8445], image_train[12234:14560], image_train[16672:18998]])
y_train = np.concatenate([image_label_train[:2326], image_label_train[2512:4838], image_label_train[6119:8445], image_label_train[12234:14560], image_label_train[16672:18998]])

In [ ]:
data_test = np.load('Dataset5_raw_test.npz')
image_test, image_label_test = data_test['image'].astype(np.float16)/ 255.0, data_test['image_label'].astype(np.int8)

In [ ]:
x_test = np.concatenate([image_test[:838], image_test[839:1677], image_test[2043:2881], image_test[4083:4921], image_test[5564:6402]])
y_test = np.concatenate([image_label_test[:838], image_label_test[839:1677], image_label_test[2043:2881], image_label_test[4083:4921], image_label_test[5564:6402]])

In [ ]:
data_val = np.load('Dataset5_raw_val.npz')
image_val, image_label_val = data_val['image'].astype(np.float16)/ 255.0, data_val['image_label'].astype(np.int8)

In [ ]:
x_val = np.concatenate([image_val[:836], image_val[837:1673], image_val[2040:2876], image_val[4079:4915], image_val[5559:6395]])
y_val = np.concatenate([image_label_val[:836], image_label_val[837:1673], image_label_val[2040:2876], image_label_val[4079:4915], image_label_val[5559:6395]])

In [ ]:
#Combining
image_total = np.concatenate([x_train, x_test, x_val])
label_total = np.concatenate([y_train, y_test, y_val])

In [ ]:
#split the data into test and train
trainX, testX, trainY, testY = train_test_split(image_total, label_total, test_size=0.30, random_state=42)
del image_train, image_label_train, image_test, image_label_test, image_val, image_label_val, image_total, label_total, x_train, x_test, x_val, y_train, y_test, y_val

In [ ]:
trainX

NameError: name 'trainX' is not defined

In [ ]:
#set the parameters
bs = 16
lr = 0.0001
shape = (224,224, 3) 
epochs = 15
class_number = 5

#define the model by mobilenetv2
def MobileNetV2_model(learning_rate, input_shape, class_number):
    baseModel = MobileNetV2(include_top=False, input_tensor=Input(shape=input_shape))
    for layer in baseModel.layers[:-4]:
        layer.trainable = False
    #baseModel.summary()
    model = Sequential()
    model.add(baseModel)
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(class_number, activation='softmax'))
    return model

#compile model
model = MobileNetV2_model(lr,shape,class_number)
model.compile(loss= "sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

In [ ]:
#start the train
# TO DO: ADDING EARLY STOPPING
print("[INFO] training ...")
H = model.fit(trainX, trainY, batch_size=bs, 
            validation_data=(testX, testY),	
            epochs=epochs)

[INFO] training ...
Epoch 1/15
875/875 [==============================] - 68s 73ms/step - loss: 1.2792 - accuracy: 0.5091 - val_loss: 0.5457 - val_accuracy: 0.8205
Epoch 2/15
875/875 [==============================] - 49s 56ms/step - loss: 0.6542 - accuracy: 0.7786 - val_loss: 0.3991 - val_accuracy: 0.8683
Epoch 3/15
875/875 [==============================] - 52s 59ms/step - loss: 0.4793 - accuracy: 0.8505 - val_loss: 0.3075 - val_accuracy: 0.8963
Epoch 4/15
875/875 [==============================] - 48s 55ms/step - loss: 0.3892 - accuracy: 0.8747 - val_loss: 0.2921 - val_accuracy: 0.9053
Epoch 5/15
875/875 [==============================] - 48s 55ms/step - loss: 0.3281 - accuracy: 0.8971 - val_loss: 0.3173 - val_accuracy: 0.8992
Epoch 6/15
875/875 [==============================] - 48s 55ms/step - loss: 0.2878 - accuracy: 0.9099 - val_loss: 0.2768 - val_accuracy: 0.9132
Epoch 7/15
875/875 [==============================] - 50s 57ms/step - loss: 0.2778 - accuracy: 0.9151 - val_loss: 0.